## Trực quan hóa Word2Vec
/> Sử dụng tensorboard để trực quan hóa word2vec.

In [1]:
import warnings
warnings.filterwarnings(action='ignore')


import tensorflow as tf
from tensorflow.contrib.tensorboard.plugins import projector
tf.logging.set_verbosity(tf.logging.ERROR)

import numpy as np
import gensim
import os

/> Load model word2vec.

In [2]:
file_name = "./model/baomoi.vn.model.bin"
model = gensim.models.keyedvectors.KeyedVectors.load_word2vec_format(file_name, binary=True)

/> Sau khi load model thành công, chúng ta cần lưu lại số từ vựng có trong model.

In [3]:
max_size = len(model.key_to_index.keys())-1

/> Chúng ta biết rằng số chiều của các word vector là $\boldsymbol{V} \times \boldsymbol{N}$. Tức số từng vựng $\boldsymbol{V} \times$ số neuron trong hidden layer $\boldsymbol{N}$. Bây giờ chúng ta sẽ tạo ma trận $\boldsymbol{W}$ và lưu vào biến `w2v`.

In [4]:
w2v = np.zeros((max_size, model.vector_size))

/> Tiếp theo, chúng ta sẽ lưu các từ cùng các word vectors vào file `metadata.tsv`.

In [5]:
if not os.path.exists('projections_vi'):
    os.makedirs('projections_vi')
    
with open("projections_vi/metadata.tsv", 'w+') as file_metadata:    
    for i, word in enumerate(model.index_to_key[:max_size]):
        # lưu embedding vector
        w2v[i] = model[word]
        
        # lưu word tương ứng 
        file_metadata.write(word + '\n')

/> Khởi tạo TensorFlow session.

In [6]:
sess = tf.InteractiveSession()

2022-02-26 22:12:56.569333: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: SSE4.1 SSE4.2 AVX AVX2 FMA
2022-02-26 22:12:56.598143: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2199995000 Hz
2022-02-26 22:12:56.598667: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x556ae83593c0 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2022-02-26 22:12:56.598707: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version
2022-02-26 22:12:56.600162: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2022-02-26 22:12:56.632820: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:983] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-02-26

/> Tạo biến `embeddings` để chứa các từ nhúng và các thiết lập cần thiết cho Tensorboard.

In [7]:
with tf.device("/gpu:0"):
    embedding = tf.Variable(w2v, trainable=False, name='embedding')

In [8]:
tf.global_variables_initializer().run()

2022-02-26 22:13:08.326839: W tensorflow/core/framework/cpu_allocator_impl.cc:81] Allocation of 1053732000 exceeds 10% of system memory.
2022-02-26 22:13:09.353171: W tensorflow/core/framework/cpu_allocator_impl.cc:81] Allocation of 1053732000 exceeds 10% of system memory.


In [9]:
saver = tf.train.Saver()

In [10]:
writer = tf.summary.FileWriter('projections_vi', sess.graph)

In [11]:
config = projector.ProjectorConfig()
embed = config.embeddings.add()

/> Chỉ định tập tin để `metadata.tsv` để trực quan hóa bằng Tensorboard.

In [12]:
embed.tensor_name = 'embedding'
embed.metadata_path = 'metadata.tsv'

/> Lưu lại model dùng để visualization của TensorBoard.

In [13]:
projector.visualize_embeddings(writer, config)
saver.save(sess, 'projections_vi/model.ckpt', global_step=max_size)

'projections_vi/model.ckpt-439055'

/> Mở terminal và nhập lệnh dưới đây để trực quan hóa.
  ```shell
  tensorboard --logdir=projections_vi --port=8000
  ```

  ![](./images/02.png)